# Unsupervised Feature Based Algorithms for Time Series Extrinsic Regression

This is the webpage and repo package to support the paper "Unsupervised Feature Based Algorithms 
for Time Series Extrinsic Regression" published in Data Mining and Knowledge Discovery.

Our results files are stored [here](https://github.com/time-series-machine-learning/tsml-eval/tree/main/tsml_eval/publications/y2023/tser_archive_expansion/results).

## Datasets

Datasets will be uploaded individually to the [timeseriesclassification.com](http://www.timeseriesclassification.com) website in due course. For now, we provide the following links:

<https://drive.google.com/file/d/1CXFof3vs6ypcqU5s5aBEZHVe2OtSVvOF/view?usp=sharing> - 63 TSER datasets

## Install

To install the latest version of the package with up-to-date algorithms, run:

    pip install tsml-eval

To install the package at the time of publication, run:

    pip install tsml-eval==0.1.0

Not all estimator dependencies are installed by default. You can install these individually as required or use the following dependency groups when installing:

    pip install tsml-eval[all_extras,deep_learning]

To install dependency versions used at the time of publication, use the publication requirements.txt:

    pip install -r tsml_eval/publications/2023/tser_archive_expansion/static_publication_reqs.txt

## Usage

### Command Line

Run [run_experiments.py](https://github.com/time-series-machine-learning/tsml-eval/blob/main/tsml_eval/publications/y2023/tser_archive_expansion/run_experiments.py) with the following arguments:

1. Path to the data directory

2. Path to the results directory

3. The name of the model to run (see [set_tser_exp_regressor.py](https://github.com/time-series-machine-learning/tsml-eval/blob/main/tsml_eval/publications/y2023/tser_archive_expansion/set_tser_exp_regressor.py), i.e. Ridge, DrCIF, InceptionE)

4. The name of the problem to run

5. The resample number to run (0 is base train/test split)

i.e. to run Covid3Month using linear regression on the base train/test split:

    python tsml_eval/publications/2023/tser_archive_expansion/run_experiments.py data/ results/ Ridge Covid3Month 0

### Using Regressors

The regressors used in our experiments extend the `scikit-learn` interface and can also be used like their estimators:

In [1]:
from aeon.regression.convolution_based import RocketRegressor
from sklearn.metrics import mean_squared_error
from tsml.datasets import load_minimal_gas_prices

from tsml_eval.estimators import SklearnToTsmlRegressor
from tsml_eval.publications.y2023.tser_archive_expansion import _set_tser_exp_regressor
from tsml_eval.utils.estimator_validation import is_sklearn_regressor

Data can be loaded using whichever method is most convenient, but should be formatted as either a 3D numpy array of shape (n_samples, n_channels, n_timesteps) or a list of length (n_samples) containing 2D numpy arrays of shape (n_channels, n_timesteps).

A function is available for loading from .ts files.

In [2]:
# load example TSER dataset
X_train, y_train = load_minimal_gas_prices("TRAIN")
X_test, y_test = load_minimal_gas_prices("TEST")

# data can be loaded from .ts files using the following function
# from tsml.datasets import load_from_ts_file
# X, y = load_from_ts_file("data/data.ts")

print(type(X_train), type(y_train))
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train[:5]

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(20, 1, 20) (20,)
(20, 1, 20) (20,)


array([[[2.39, 2.36, 2.45, 2.45, 2.54, 2.69, 2.63, 2.73, 2.7 , 2.73,
         2.68, 2.76, 2.76, 2.68, 2.39, 2.4 , 2.36, 2.6 , 2.77, 2.76]],

       [[3.02, 3.09, 3.01, 3.01, 2.98, 3.11, 3.13, 3.13, 3.23, 3.36,
         3.31, 3.25, 3.24, 3.23, 3.15, 3.21, 3.36, 3.3 , 3.4 , 3.62]],

       [[3.95, 4.03, 4.07, 4.35, 4.25, 4.33, 4.45, 4.65, 4.77, 4.71,
         4.66, 4.97, 5.13, 5.21, 5.39, 5.66, 5.52, 5.32, 5.25, 4.96]],

       [[2.62, 2.63, 2.63, 2.71, 2.71, 2.72, 2.58, 2.58, 2.58, 2.62,
         2.61, 2.67, 2.57, 2.59, 2.63, 2.59, 2.59, 2.5 , 2.55, 2.55]],

       [[2.44, 2.43, 2.47, 2.48, 2.5 , 2.57, 2.65, 2.62, 2.63, 2.75,
         2.76, 2.72, 2.77, 2.79, 2.73, 2.91, 2.98, 2.91, 2.86, 2.96]]])

Regressors can be built using the `fit` method and predictions can be made using `predict`.

In [3]:
# build a ROCKET regressor and make predictions
rocket = RocketRegressor(n_kernels=1000, random_state=0)
rocket.fit(X_train, y_train)
rocket.predict(X_test)

array([-0.39051194, -0.32556358, -0.26716222, -0.29462747, -0.38713609,
       -0.35210252, -0.35538663, -0.34188738, -0.386536  , -0.38444336,
       -0.28417025, -0.34912216, -0.3575563 , -0.29534897, -0.3730631 ,
       -0.40209279, -0.31955412, -0.34667809, -0.37519843, -0.38101579])

Here we run some of the regressors from the publication and find the RMSE for them on our example dataset.

In [4]:
regressors = [
    "1NN-DTW",
    "1NN-ED",
    "RandF",
    "ROCKET",
]

rmse = []
for regressor_name in regressors:
    # Select a regressor by name, see set_tser_exp_regressor.py for options
    regressor = _set_tser_exp_regressor(regressor_name, random_state=0)

    # if it is a sklearn regressor, wrap it to work with time series data
    if is_sklearn_regressor(regressor):
        regressor = SklearnToTsmlRegressor(
            regressor=regressor, concatenate_channels=True, random_state=0
        )

    # fit and predict
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    rmse.append(mean_squared_error(y_test, y_pred, squared=False))

rmse

[0.0807299822065842,
 0.08058022963859234,
 1.3423714791418515,
 0.0742857643859832,
 0.10120786111499722]